In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os 
os.chdir('/content/drive/My Drive/Mestrado/2021.2/Aprendizagem de Maquina/trabalho_02/')

Mounted at /content/drive


# Imports

In [2]:
from sklearn.datasets import load_iris
import pandas as pd
import seaborn as sns
import numpy as np
import pandas as pd
import math
import scipy
from mpl_toolkits import mplot3d
from matplotlib.collections import PolyCollection
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import colors as mcolors
from IPython.display import display

import matplotlib.pyplot as plt

In [3]:
from src.utils import *

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
def transform(x):
    if x == 'setosa':
        return 0
    elif x == 'virginica':
        return 1
    elif x == 'versicolor':
        return 2

In [5]:
df_iris = sns.load_dataset('iris')
df_iris['target'] = df_iris['species'].apply(lambda x: transform(x))
df_iris.drop(['species'], axis=1, inplace=True)

In [6]:
df_iris.head(3)

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


# Classificador

In [8]:
class ClassifierBayes():
    
    def __init__(self, discrimante='quadratico', type_cov=None):
        self.discrimante = discrimante
        if self.discrimante == 'linear':
            self.type_cov = type_cov

    def fit(self, X_train, y_train, verbose=False):
        df_train = X_train.copy()
        df_train['target'] = y_train
        self.list_class = df_train['target'].unique()
        self.features = [col for col in df_train.columns if col != 'target']
    
        self.dict_priori = {}
        self.dict_df_class = {}
        self.dict_mean = {}
        self.dict_cov = {}
        self.dict_inv_cov = {}
        self.dict_det = {}
        self.is_singular = False
        
        for j in sorted(self.list_class):
            self.dict_priori[j] = (len(df_train.loc[df_train['target'] == j]) / len(df_train))
            self.dict_df_class[j] = df_train[df_train['target'] == j].copy()
            self.dict_mean[j] = np.matrix([self.dict_df_class[j][col].mean() for col in self.dict_df_class[j].columns if col != 'target'])
            if self.discrimante == 'quadratico':
                mtx_cov = np.matrix(self.dict_df_class[j].drop(['target'], axis=1).cov().values)
                self.dict_cov[j] = mtx_cov
                self.dict_inv_cov[j] = self.__inv_mtx(self.dict_cov[j])
                self.dict_det[j] = self.__det_mtx(self.dict_cov[j])
        
        if (self.discrimante == 'linear'):
            cov_linear = None
            if(self.type_cov == 'pool'):
                cov_linear = sum([self.dict_priori[j] * np.matrix(self.dict_df_class[j].drop(['target'], axis=1).cov().values) for j in sorted(self.list_class)])
            elif (self.discrimante == 'linear') & (self.type_cov == 'all_data'):
                cov_linear = np.matrix(df_train.drop(['target'], axis=1).cov().values)
            for j in sorted(self.list_class):
                self.dict_cov[j] = cov_linear
                self.dict_inv_cov[j] = self.__inv_mtx(self.dict_cov[j])
                self.dict_det[j] = self.__det_mtx(self.dict_cov[j])
        if verbose:
            print('media: ', self.dict_mean[j]) 
            print('inversa mtx covariancia: ', self.dict_inv_cov) 
            print('determinante: ',self.dict_det) 

    def __inv_mtx(self, cov):
        try:
            return cov.I
        except np.linalg.LinAlgError as err:
            self.is_singular = True
            return np.linalg.pinv(cov)
    
    def __det_mtx(self, cov):
        if self.is_singular:
            eig_values = np.linalg.eig(cov)
            return np.product([i for i in eig_values[0] if i > 1e-12])
        else:
            return np.linalg.det(cov)
    
    def predict(self, df, verbose=False):

        list_predict = []
        for idx, row in df.iterrows():
    
            x = np.matrix([df[self.features].iloc[idx].tolist()])
            dict_discriminating = {}
            max_disc = -np.inf
            class_predict = -np.inf
            for j in sorted(self.list_class):
                if verbose:
                    print('x.T', x.T)
                    print('inv_cov', self.dict_inv_cov[j])
                    print('x', x)
                    print('1º', -0.5 * float(x * self.dict_inv_cov[j] * x.T))
                    print('2º', 0.5 * float(self.dict_mean[j] * self.dict_inv_cov[j] * x.T))
                    print('3º', 0.5 * float(self.dict_mean[j] * self.dict_inv_cov[j] * self.dict_mean[j].T))
                    print('4º', 0.5 * float(x * self.dict_inv_cov[j] * self.dict_mean[j].T))
                    print('5º', np.log(self.dict_priori[j]))
                    print('6º', -0.5 * np.log(2 * np.pi) - 0.5 * np.log(self.dict_det[j]))
                if self.discrimante == 'linear':
                    dict_discriminating[j] = (
                        (0.5 * float(self.dict_mean[j] * self.dict_inv_cov[j] * x.T)) - # 1 º
                        (0.5 * float(self.dict_mean[j] * self.dict_inv_cov[j] * self.dict_mean[j].T)) + # 2 º
                        (0.5 * float(x * self.dict_inv_cov[j] * self.dict_mean[j].T)) + # 3 º
                        np.log(self.dict_priori[j])
                    )
                elif self.discrimante == 'quadratico':
                    dict_discriminating[j] = (
                        (-0.5 * float(x * self.dict_inv_cov[j] * x.T)) + # 1º
                        (0.5 * float(self.dict_mean[j] * self.dict_inv_cov[j] * x.T)) - # 2 º
                        (0.5 * float(self.dict_mean[j] * self.dict_inv_cov[j] * self.dict_mean[j].T)) + # 3 º
                        (0.5 * float(x * self.dict_inv_cov[j] * self.dict_mean[j].T)) + # 4 º
                        np.log(self.dict_priori[j]) + # 5º
                        ((-0.5 * (np.log(2 * np.pi)) - (0.5 * np.log(self.dict_det[j])))) #ci

                    )

                if dict_discriminating[j] > max_disc:
                    max_disc = dict_discriminating[j]
                    class_predict = j
            if verbose:
                print(f'Linha {idx} discriminantes: ')
                print(dict_discriminating)
            list_predict.append(class_predict)
        
        return list_predict

In [9]:
X_train, y_train, X_test, y_test = split_train_test(df_iris.copy(), col_target='target', train_size=0.8, stratify=True)

In [10]:
quadratico = ClassifierBayes()

quadratico.fit(X_train, y_train)

In [11]:
X_test['predict'] = quadratico.predict(X_test.copy())

In [12]:
X_test['target'] = y_test

In [13]:
def accuracy(y_true, y_pred): # Taxa de acerto
    
    qtd_acertos = 0
    for true, pred in zip(y_true, y_pred):
        if true == pred: 
            qtd_acertos += 1

    return qtd_acertos / len(y_true)

In [14]:
accuracy(X_test['target'], X_test['predict'])

0.9666666666666667

# Linear

In [23]:
X_train, y_train, X_test, y_test = split_train_test(df_iris.copy(), col_target='target', train_size=0.8, stratify=True)

In [24]:
linear_pool = ClassifierBayes(discrimante='linear', type_cov='pool')
linear_all_data = ClassifierBayes(discrimante='linear', type_cov='all_data')

linear_pool.fit(X_train.copy(), y_train)
linear_all_data.fit(X_train.copy(), y_train)

In [25]:
X_test['predict_all_data'] = linear_all_data.predict(X_test.copy())
X_test['predict_pool'] = linear_pool.predict(X_test.copy())

In [26]:
X_test['target'] = y_test

In [27]:
accuracy(X_test['target'], X_test['predict_all_data'])

0.8666666666666667

In [28]:
accuracy(X_test['target'], X_test['predict_pool'])

1.0